# Kinectic Command: Gesture Recognition Model Training

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

## 1. Data Loading and Preparation

In [ ]:
def generate_synthetic_gesture_data(num_samples_per_gesture=100, num_features=42, num_gestures=4):
    """Generates synthetic gesture data (landmark features + labels)."""
    np.random.seed(42)
    gestures = ['fist', 'open_palm', 'pointing', 'ok']
    if num_gestures > len(gestures):
        raise ValueError(f"Requested {num_gestures} gestures, but only {len(gestures)} are defined.")
    
    selected_gestures = gestures[:num_gestures]
    
    X = []
    y = []
    
    # Create distinct means for each gesture's features
    means = [np.random.rand(num_features) * (i + 1) for i in range(num_gestures)]
    
    for i, gesture in enumerate(selected_gestures):
        # Generate samples around the mean for this gesture
        samples = means[i] + np.random.randn(num_samples_per_gesture, num_features) * 0.5 # Add noise
        X.extend(samples)
        y.extend([gesture] * num_samples_per_gesture)
        
    X = np.array(X)
    y = np.array(y)
    
    # Shuffle the data
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    
    return X, y, selected_gestures

In [ ]:
# Generate or load data
# In a real scenario, load pre-processed landmark data here
X, y, gesture_labels = generate_synthetic_gesture_data(num_samples_per_gesture=200, num_features=42, num_gestures=4)

print(f"Generated data shape: X={X.shape}, y={y.shape}")
print(f"Gesture labels: {gesture_labels}")
print(f"Example features (first sample):\n{X[0][:10]}...") # Print first 10 features of first sample
print(f"Example label (first sample): {y[0]}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData scaled using StandardScaler.")
print(f"Example scaled features (first training sample):\n{X_train_scaled[0][:10]}...")

## 2. Model Definition and Training (Initial)

In [ ]:
# Define a simple SVM model
initial_model = SVC(random_state=42)

# Train the initial model
initial_model.fit(X_train_scaled, y_train)

print("Initial SVM model trained.")

# Evaluate the initial model (optional step here, full evaluation later)
y_pred_initial = initial_model.predict(X_test_scaled)
initial_accuracy = accuracy_score(y_test, y_pred_initial)
print(f"Initial model accuracy on test set: {initial_accuracy:.4f}")

## 3. Hyperparameter Tuning

In [ ]:
# Define the parameter grid for SVC
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

print("Defined parameter grid for GridSearchCV:")
print(param_grid)

In [ ]:
# Perform GridSearchCV
# Use a smaller subset for faster tuning if needed, otherwise use full X_train_scaled
grid_search = GridSearchCV(SVC(random_state=42), param_grid, refit=True, verbose=1, cv=5, n_jobs=-1)

print("\nStarting GridSearchCV...")
grid_search.fit(X_train_scaled, y_train)
print("GridSearchCV finished.")

In [ ]:
# Print best parameters and best score
print(f"\nBest parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score (accuracy): {grid_search.best_score_:.4f}")

# The best model is automatically refit on the entire training set
best_model = grid_search.best_estimator_

## 4. Model Evaluation

In [ ]:
# Make predictions on the test set using the best model
y_pred_tuned = best_model.predict(X_test_scaled)

# Calculate final accuracy
final_accuracy = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned model accuracy on test set: {final_accuracy:.4f}")

In [ ]:
# Print classification report
print("\nClassification Report:")
report = classification_report(y_test, y_pred_tuned, target_names=gesture_labels)
print(report)

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred_tuned, labels=gesture_labels)
print("\nConfusion Matrix:")
print(cm)

## 5. Results Visualization

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=gesture_labels, yticklabels=gesture_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Gesture Recognition')
plt.show()

## 6. Save Model and Scaler

In [ ]:
# Define file paths
model_filename = 'gesture_model.joblib'
scaler_filename = 'scaler.joblib'
labels_filename = 'gesture_labels.joblib'

# Save the trained model
joblib.dump(best_model, model_filename)
print(f"Trained model saved to {model_filename}")

# Save the scaler
joblib.dump(scaler, scaler_filename)
print(f"Scaler saved to {scaler_filename}")

# Save the gesture labels
joblib.dump(gesture_labels, labels_filename)
print(f"Gesture labels saved to {labels_filename}")

### End of Notebook
The trained model (`gesture_model.joblib`), the scaler (`scaler.joblib`), and the gesture labels (`gesture_labels.joblib`) are now saved and can be loaded by the main Kinectic Command application for real-time gesture recognition.